# Mixed strategy Nash EQ with maxmin/linprog

It is possible to solve for the mixed strategy Nash EQ in a zero sum game using linear programming by using duality and recasting the problem. 

In [1]:
import numpy as np 
import nashpy # install using "pip install nashpy"
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.optimize import linprog 

In [2]:
def print_payoffs(U, A): 
    '''print_payoffs: Nicely formatted for a 2*2 game 
        INPUTS: 
            U1,U2: (matrices, dim=na1*na2) Payoffs 
            A1: (list of str, len=na1) List of actions of player 1
            A2: (list of str, len=na2) list of actions of player 2
        
        OUTPUT:
            tab: pandas dataframe, na1*na2 with payoff tuples 
    '''
    assert len(U) == 2, f'only implemented for 2-player games'
    assert len(A) == 2, f'only implemented for 2-player games'

    U1 = U[0]
    U2 = U[1]
    A1 = A[0]
    A2 = A[1]

    na1,na2 = U1.shape
    assert len(A1) == na1
    assert len(A2) == na2

    # "matrix" of tuples 
    X = [[(U1[r,c],U2[r,c]) for c in range(na2)] for r in range(na1)]

    # dataframe version 
    tab = pd.DataFrame(X, columns=A2, index=A1)
    
    return tab 

In [3]:
def solve_zerosum_with_linprog(U):
    '''solve_zerosum_with_linprog(): Solve a zero sum game using linear programming
    
        INPUT: U (k*k square matrix), payoffs in zero sum game (opponent gets -U.T)
        OUTPUT: alpha (k-vector) of probability weights for each action (the symmetric equilibrium)
    '''
    k, k2 = U.shape
    assert k == k2, f'Input matrix must be square, got {k}*{k2}'

    oo = np.zeros((1,k))
    ii = np.ones((1,k))

    # objective: c = [-1, 0, 0, ..., 0]
    c = np.insert(oo, 0, -1.0) # insert -1 in front (pos = index 0)
    
    # inequality constraints A*x <= b
    # top = [ 1 ...
    #         1 -1*A.Tl
    #         1  ...  ]
    # bot = [ 0 -1 0 0 
    #         0 0 -1 0 
    #         0 0 0 -1]
    top  = np.hstack( (ii.T, -1*U.T) )
    bot  = np.hstack( (oo.T, -1*np.eye(k)) )
    A_ub = np.vstack((top, bot))
    
    b_ub = np.zeros((1, 2*k))
    b_ub = np.matrix(b_ub)
    
    # contraints Ax = b
    # A = [0, 1, 1, ..., 1]
    A = np.matrix(np.hstack((0, np.ones((k,)))))
    b = 1.0 # just one condition so scalar 

    # v and alpha must be non-negative
    bounds = [(0,None) for i in range(k+1)]

    # call the solver
    sol = linprog(c=c, A_ub=A_ub, b_ub=b_ub, A_eq=A, b_eq=b)
    
    # remove the first element: just return the Nash EQ 
    alpha = sol.x[1:]
    return alpha

# Sten saks papir

In [4]:
U1 = np.array([[0,1,-1], [-1,0,1], [1,-1,0]])
U2 = -U1
A1 = ['sten', 'saks','papir']
A2 = A1

In [5]:
print_payoffs([U1, U2], [A1, A2])

,sten,saks,papir
sten,"(0, 0)","(1, -1)","(-1, 1)"
saks,"(-1, 1)","(0, 0)","(1, -1)"
papir,"(1, -1)","(-1, 1)","(0, 0)"


In [6]:
sol = nashpy.Game(U1,U2).support_enumeration()
list(sol)

[(array([0.33333333, 0.33333333, 0.33333333]),
  array([0.33333333, 0.33333333, 0.33333333]))]

Solve with linear programming

In [7]:
sol = solve_zerosum_with_linprog(U1)
sol

array([0.33333333, 0.33333333, 0.33333333])

An asymmetric zero sum game

solve with `nasphy`

In [8]:
%%timeit
sol = nashpy.Game(U1,U2).support_enumeration()
list(sol)

The slowest run took 4.34 times longer than the fastest. This could mean that an intermediate result is being cached.
18.7 ms ± 9.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Solving with `maximin` 

In [9]:
%%timeit 
sol = solve_zerosum_with_linprog(U1)
sol

4.49 ms ± 728 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# A larger game

Wi simply copy the came for times 

In [10]:
U1 = np.tile(U1, (2,2)) # copy twice in both x and y direction
U2 = -U1

A1 = ['sten', 'saks','papir']
A1 = [f'{a}{i}' for i in range(2) for a in A1 ]

print_payoffs([U1, U2], [A1, A1])

,sten0,saks0,papir0,sten1,saks1,papir1
sten0,"(0, 0)","(1, -1)","(-1, 1)","(0, 0)","(1, -1)","(-1, 1)"
saks0,"(-1, 1)","(0, 0)","(1, -1)","(-1, 1)","(0, 0)","(1, -1)"
papir0,"(1, -1)","(-1, 1)","(0, 0)","(1, -1)","(-1, 1)","(0, 0)"
sten1,"(0, 0)","(1, -1)","(-1, 1)","(0, 0)","(1, -1)","(-1, 1)"
saks1,"(-1, 1)","(0, 0)","(1, -1)","(-1, 1)","(0, 0)","(1, -1)"
papir1,"(1, -1)","(-1, 1)","(0, 0)","(1, -1)","(-1, 1)","(0, 0)"


solve with `nasphy` (solution not actually stored when we use `timeit`) 

In [11]:
%%timeit
sol = nashpy.Game(U1,U2).support_enumeration()
list(sol) # we need to materialize the solution for computation to take place

/Users/mrz577/opt/anaconda3/lib/python3.8/site-packages/nashpy/algorithms/support_enumeration.py:196: RuntimeWarning: 
An even number of (28) equilibria was returned. This
indicates that the game is degenerate. Consider using another algorithm
to investigate.
                  
  warnings.warn(warning, RuntimeWarning)


1.09 s ± 89.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Solving with `maximin` 

In [12]:
%%timeit 
sol = solve_zerosum_with_linprog(U1)
sol

4.58 ms ± 770 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
